In [1]:
%pip install generative-ai-hub-sdk panda hana-ml
import os

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 25.0.1
[notice] To update, run: C:\Users\GPER\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


<b>Loading vector data from a csv file</b><br/>
Download the following csv file and save it in your system. Download File<br/>
Disclaimer : To execute the tutorial if you are uploading sample CSV FILE, Use
steps 1-5 to upload the file Don’t use Hana Data base Explorer to directly
upload the file as it may change the format in HANA.<br/><br/>
Execute the following python code in the same folder. This will load the data and store it in a data-frame.

In [1]:
import pandas as pd
df = pd.read_excel('AGILITA_Products.xlsx', dtype=str, header=1)
df["ARTICLENR"] = df['Material']
df["NAME"] = df['Price List Item']
df["VECTOR_STR"] = df['Unnamed: 2']
df = df[["ARTICLENR", "NAME", "VECTOR_STR"]]
df = df.dropna(subset=["ARTICLENR"])

df.head(3)

,ARTICLENR,NAME,VECTOR_STR
1,8011437,SAP Signavio Process Manager,Nutzen Sie die Macht der Prozesse mit einer in...
2,8011439,SAP Signavio Process Governance,Durch die gemeinsame Dokumentation von Prozess...
3,8011440,SAP Signavio Process Collaboration Hub,Zusammenarbeit im Prozess und Transparenz\nIhr...


<b>Connection to the HANA Vector store</b><br/>
Execute the following python code to create a connection to the HANA Vector storage.

In [2]:
import os
from hana_ml import ConnectionContext
# cc = ConnectionContext(userkey='VDB_BETA', encrypt=True)
cc = ConnectionContext(
    address=os.environ.get("DB_ADDRESS"),
    port=os.environ.get("DB_PORT"),
    user=os.environ.get("DB_USER"),
    password=os.environ.get("DB_PASSWORD"), 
    encrypt=True
) 

print(cc.hana_version())
print(cc.get_current_schema())



4.00.000.00.1727771918 (fa/CE2024.28)
381993CF47F34B458C0588F6AF77BB6B_BYSRFQOXOP7AF5447PYVCPLYO_RT


<b>Uploading the data to the database</b><br/>
TExecute the following code to upload the data to the database.

In [3]:
from hana_ml.dataframe import create_dataframe_from_pandas
v_hdf = create_dataframe_from_pandas(
    connection_context=cc,
    pandas_df=df,
    table_name="381993CF47F34B458C0588F6AF77BB6B.CH_AGILITA_AISALES_EXTERNAL_MATERIALVECTORS",
    allow_bigint=True,
    append=True
    )

100%|██████████| 1/1 [00:00<00:00, 23.89it/s]


<b>Get Embeddings</b><br/>
Define the function get_embedding() to generate embeddings for our input texts. Execute the following python code.

In [5]:
# pip install python-dotenv

In [8]:
import os
# Create a dictionary with the environment variables you want to set.
env_vars = {
    'AICORE_AUTH_URL' : os.environ.get("AICORE_AUTH_URL"),
    'AICORE_CLIENT_ID' : os.environ.get("AICORE_CLIENT_ID"),
    'AICORE_CLIENT_SECRET' : os.environ.get("AICORE_CLIENT_SECRET"),
    'AICORE_BASE_URL' : os.environ.get("AICORE_BASE_URL"),
    'AICORE_RESOURCE_GROUP' : 'default'
}
print(env_vars)

# Set the environment variables using `os.environ`.
for key, value in env_vars.items():
    os.environ[key] = value

{'AICORE_AUTH_URL': 'https://agilitaesres.authentication.eu10.hana.ondemand.com/oauth/token', 'AICORE_CLIENT_ID': 'sb-221be585-00af-423b-b21c-dbb68b44522a!b509560|aicore!b540', 'AICORE_CLIENT_SECRET': 'addf53da-d6a0-4f7b-afeb-979fa4a47243$mZGC-aQA0FKvK8IhmnDR38kJpt2W9N-J1wHWxyMbjbM=', 'AICORE_BASE_URL': 'https://api.ai.prod.eu-central-1.aws.ml.hana.ondemand.com', 'AICORE_RESOURCE_GROUP': 'default'}


In [9]:
# Get embeddings
from gen_ai_hub.proxy.native.openai import embeddings

def get_embedding(input, model="text-embedding-3-small") -> str:
    response = embeddings.create(
        model_name=model,
        input=input
    )
    return response.data[0].embedding

In [10]:
embeddings_list = []

for index, row in df.iterrows():
    vector_str = row['VECTOR_STR']  # The text you want to get embedding for
    embedding = get_embedding(vector_str)  # Fetch embedding for the current VECTOR_STR
    embeddings_list.append(embedding)  # Store the embedding in the list

# Step 2: Add the embeddings as a new column to the DataFrame
df['Embedding'] = embeddings_list

print("Embeddings added successfully:")
print(embedding[:10])

Embeddings added successfully:
[-0.029244810342788696, 0.00618803221732378, 0.0529797300696373, 0.011104551143944263, -0.004256921354681253, -0.014304527081549168, -0.009165492840111256, 0.03428848087787628, 0.02084222622215748, -0.00015720078954473138]


<b>Creating vectors from strings</b><br/>
The vectors for the strings can be created using the TO_REAL_VECTOR() function. <br/>
Execute the following code to update the VECTOR column with the vectors.

In [9]:
import json

# database connection
conn = cc.connection
cursor = conn.cursor()

# Process each row 
for index, row in df.iterrows():
    vector_str = row['VECTOR_STR']
    embedding = get_embedding(vector_str)  # Generate embedding 

    if not embedding:
        print(f"No embedding generated for {row['ARTICLENR']}")
        continue  # Skip this row

    # Convert embedding list to json string for correct format
    embedding_str = json.dumps(embedding)

    # Check if ARTICLENR exists
    cursor.execute("""
        SELECT COUNT(*) FROM 381993CF47F34B458C0588F6AF77BB6B.CH_AGILITA_AISALES_EXTERNAL_MATERIALVECTORS
        WHERE ARTICLENR = :articlenr
    """, {'articlenr': row["ARTICLENR"]})
    
    result = cursor.fetchone()
    
    if result[0] > 0:  # If row exists, UPDATE
        sql_command = """
            UPDATE 381993CF47F34B458C0588F6AF77BB6B.CH_AGILITA_AISALES_EXTERNAL_MATERIALVECTORS
            SET NAME = :name, VECTOR = TO_REAL_VECTOR(CAST(:vector AS NVARCHAR))
            WHERE ARTICLENR = :articlenr
        """
        cursor.execute(sql_command, {'name': row["NAME"], 'vector': embedding_str, 'articlenr': row["ARTICLENR"]})
    
    else:  # If row does not exist, INSERT
        sql_command = """
            INSERT INTO 381993CF47F34B458C0588F6AF77BB6B.CH_AGILITA_AISALES_EXTERNAL_MATERIALVECTORS
            (ARTICLENR, NAME, VECTOR) 
            VALUES (:articlenr, :name, TO_REAL_VECTOR(CAST(:vector AS NVARCHAR)))
        """
        cursor.execute(sql_command, {'articlenr': row["ARTICLENR"], 'name': row["NAME"], 'vector': embedding_str})

# Commit changes and close the connection
conn.commit()
cursor.close()
conn.close()

<b>Running vector search</b><br/>
Define a function run_vector_search(). This function will search the vector database and finds the rows which are most similar to a given query.<br/>
Execute the following python code

In [11]:
def run_vector_search(query: str, metric="COSINE_SIMILARITY", k=4):

    # Determine the sorting order based on the metric
    if metric == 'L2DISTANCE':
        sort = 'ASC'  # Lower distance is better for L2 (Euclidean)
    else:
        sort = 'DESC'  # Higher similarity is better for COSINE_SIMILARITY

    # Generate the embedding for the query
    query_vector = get_embedding(query)

    # Convert the embedding to a comma-separated string and wrap with square brackets
    query_vector_str = f"[{','.join(map(str, query_vector))}]"

    # Construct the SQL query using string formatting
    sql = f'''
    SELECT TOP {k} "ARTICLENR", "NAME", "VECTOR", 
           COSINE_SIMILARITY(
               "VECTOR",  -- Use the REAL_VECTOR column directly
               TO_REAL_VECTOR('{query_vector_str}') 
           ) AS similarity
    FROM "381993CF47F34B458C0588F6AF77BB6B"."CH_AGILITA_AISALES_EXTERNAL_MATERIALVECTORS"
    ORDER BY similarity {sort}
    '''

    try:
        # Execute the query using the database connection
        hdf = cc.sql(sql)

        # Collect the results into a pandas DataFrame
        df_context = hdf.head(k).collect()

        # Return the results
        return df_context

    except Exception as e:
        print(f"An error occurred: {e}")
        print(f"SQL Query: {sql}")
        return None


In [12]:
query = "How can I run a shortest path algorithm?"
df_context = run_vector_search(query=query, metric="COSINE_SIMILARITY", k=4)
print(df_context)


  ARTICLENR                                      NAME  \
0   8014823         SAP Signavio Process Intelligence   
1   8018011  SAP Field Service Management, SME option   
2   8015529              SAP Field Service Management   
3         7                             AGIL AI.Sales   

                                              VECTOR  SIMILARITY  
0  [-0.030233390629291534, 0.02739836275577545, 0...    0.225808  
1  [-0.006160595919936895, 0.02896769717335701, 0...    0.197500  
2  [-0.006163731683045626, 0.02895912528038025, 0...    0.197488  
3  [-0.026103109121322632, 0.013193110935389996, ...    0.189498  


In [13]:
from gen_ai_hub.proxy.native.openai import chat

# Define your system content for the LLM
sys_content = '''Your task is to answer the question using the provided context wrapped in triple quotes.
If the provided context does not contain the information needed to answer this question then come up with your own answer.'''

def ask_llm(query: str, retrieval_augmented_generation: bool, metric='COSINE_SIMILARITY', k=4) -> str:
    context = ''
    
    if retrieval_augmented_generation:
        # Perform vector search to retrieve context
        df_context = run_vector_search(query, metric, k)
        
        # Use the 'NAME' column for context
        if 'NAME' in df_context.columns:
            context = ' '.join(df_context['NAME'].astype('string'))
        else:
            # In case 'NAME' does not exist, fallback to 'ARTICLENR'
            context = ' '.join(df_context['ARTICLENR'].astype('string'))
    
    user_content = '"""' + context + '"""' + ' Question: ' + query
    
    # Prepare the messages list with system and user content
    messages = [
        {"role": "system", "content": sys_content},
        {"role": "user", "content": user_content}
    ]
    
    # Make the request to the LLM (using GPT-3.5 turbo model here)
    kwargs = dict(model_name='gpt-35-turbo', messages=messages)
    
    try:
        # Request to the OpenAI API
        response = chat.completions.create(**kwargs)

        # Check if the response contains 'choices' and is a valid list
        if hasattr(response, 'choices') and len(response.choices) > 0:
            # Directly access the 'content' from the first choice
            answer = response.choices[0].message.content if hasattr(response.choices[0], 'message') else "No message found in choices."
        else:
            answer = "No valid response found in choices."
    except Exception as e:
        # In case of an error, print the error message
        print("Error during API call:", str(e))
        answer = "An error occurred while querying the LLM."
    
    return answer

In [14]:
query = "What IS SAP?"
response = ask_llm(query=query, retrieval_augmented_generation=True, metric="COSINE_SIMILARITY", k=4)
print("Answer:", response)

Answer: "SAP" stands for Systems, Applications, and Products in Data Processing. It is a multinational software corporation that develops enterprise software to manage business operations and customer relations. SAP offers a range of products, including the SAP Integration Suite, which provides capabilities for integrating various systems and applications within an organization.
